## Random Forest Classifier using pipeline and hyperparameter tuning

In [1]:
import seaborn as sns 
df= sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
df.isnull().sum()


total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [3]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [4]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

df['time']=encoder.fit_transform(df['time'])

In [6]:
df.time.unique()

array([0, 1])

In [7]:
#independent and dependent features

x=df.drop(labels=["time"],axis=1)
y=df.time

In [8]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [9]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [13]:
## feature engineering automation

#numerical pipeline

num_pipeline=Pipeline(
    
    steps=[
        
        ('imputer',SimpleImputer(strategy='median')), #missing values 
        ('scaler',StandardScaler()) # feature scaling
    
    ]
)

cat_pipeline=Pipeline(
    
    steps=[
        
        ('imputer',SimpleImputer(strategy='most_frequent')), #missing values
        ('onehotencoder',OneHotEncoder()) ## categorical feature to numerical feature
        
    
    
    ]


)


In [14]:
processor=ColumnTransformer([
    
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)



])

In [15]:
x_train=processor.fit_transform(x_train)
x_test=processor.fit_transform(x_test)

In [16]:
## Model training automation

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [18]:
# model training automation 
models={
    
    'Random forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision tree':DecisionTreeClassifier()

}

In [19]:
from sklearn.metrics import accuracy_score

In [23]:
def evaluate_model(x_train,y_train,x_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(x_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report


In [25]:
evaluate_models(x_train,y_train,x_test,y_test,models)

{'Random forest': 0.9795918367346939}

In [27]:
classifier=RandomForestClassifier()

In [34]:
#Hyperparameter tuning
params={
    
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']

}


In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=2)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ..criterion=entropy, max_depth=10, n_estimators=200; total time=   0.4s
[CV] END ..criterion=entropy, max_depth=10, n_estimators=200; total time=   0.4s
[CV] END ..criterion=entropy, max_depth=10, n_estimators=200; total time=   0.4s
[CV] END ..criterion=entropy, max_depth=10, n_estimators=200; total time=   0.4s
[CV] END ..criterion=entropy, max_depth=10, n_estimators=200; total time=   0.4s
[CV] END ...criterion=entropy, max_depth=5, n_estimators=200; total time=   0.4s
[CV] END ...criterion=entropy, max_depth=5, n_estimators=200; total time=   0.4s
[CV] END ...criterion=entropy, max_depth=5, n_estimators=200; total time=   0.4s
[CV] END ...criterion=entropy, max_depth=5, n_estimators=200; total time=   0.4s
[CV] END ...criterion=entropy, max_depth=5, n_estimators=200; total time=   0.4s
[CV] END ...criterion=gini, max_depth=None, n_estimators=300; total time=   0.6s
[CV] END ...criterion=gini, max_depth=None, n_es

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=2)

In [37]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 10, 'criterion': 'entropy'}

NameError: name 'rfc' is not defined